## 09 함수를 더 편리하게

### colab 접속

In [1]:
if __name__ == '__main__':
  !pip install import_ipynb
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive/밑바닥부터 시작하는 딥러닝3

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp37-none-any.whl size=2976 sha256=5d0b4fe25beef7b758a4397b7b38e4ae48007ae32ee37c846dc689b805e6c630
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
Mounted at /content/drive
/content/drive/MyDrive/밑바닥부터 시작하는 딥러닝3


In [2]:
import numpy as np
import import_ipynb
from Deep08 import Variable, Square, Exp


importing Jupyter notebook from Deep08.ipynb


### 9.1 파이썬 함수로 이용하기
+ 두 줄을 쓰는 것이 좋지 않으니 하나의 함수로 만들자

In [3]:
if __name__ == '__main__':
  x = Variable(np.array(0.5))
  f = Square()
  y = f(x)

In [4]:
def square(x):
  f = Square()
  return f(x)

def exp(x):
  f = Exp()
  return f(x)

In [5]:
if __name__ == '__main__':
  x = Variable(np.array(0.5))
  a = square(x)
  b = exp(a)
  y = square(b)

  y.grad = np.array(1.0)
  y.backward()
  print(x.grad)

3.297442541400256


In [6]:
if __name__ == '__main__':
  x = Variable(np.array(0.5))
  y = square(exp(square(x)))
  y.grad = np.array(1.0)
  y.backward()
  print(x.grad)

3.297442541400256


### 9.2 backward 메서드 간소화

In [ ]:
# class Variable:
#   def __init__(self, data) :
#     self.data = data
#     self.grad = None
#     self.creator = None

#   def set_creator(self, func):
#     self.creator = func
  
#   def backward(self):
#     funcs = [self.creator]
#     while funcs:
#       f = funcs.pop()
#       x, y = f.input, f.output
#       x.grad = f.backward(y.grad)

#       if x.creator is not None:
#         funcs.append(x.creator)

      

In [11]:
class Variable():
  def __init__(self, data) :
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func):
    self.creator = func

  def backward(self):
    # 만약 가장 처음 이라면 1 즉 y=np.array(1.0)을 삭제하기위해 추가된 코드
    if self.grad is None:
      self.grad = np.ones_like(self.data)
    
    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      x,y = f.input, f.output
      x.grad = f.backward(y.grad)

      if x.creator is not None :
        funcs.append(x.creator)

In [13]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx
def square(x):
  f = Square()
  return f(x)

def exp(x):
  f = Exp()
  return f(x)

In [15]:
if __name__ == '__main__':
  x = Variable(np.array(0.5))
  y = square(exp(square(x)))
  y.backward()
  print(x.grad)

3.297442541400256


### 9.3 ndarray만 취급하기
+ 지금은 ndarray만을 취급하고 있는데 float 등을 넣었을 때의 상황을 고려

In [16]:
class Variable():
  def __init__(self, data) :

    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError(f'{type(data)}은 지원하지 않습니다.')

    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func):
    self.creator = func

  def backward(self):
    # 만약 가장 처음 이라면 1 즉 y=np.array(1.0)을 삭제하기위해 추가된 코드
    if self.grad is None:
      self.grad = np.ones_like(self.data)
    
    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      x,y = f.input, f.output
      x.grad = f.backward(y.grad)

      if x.creator is not None :
        funcs.append(x.creator)

In [17]:
if __name__ == '__main__':
  x = Variable(np.array(1.0))
  x = Variable(None)
  x = Variable(1.0)

TypeError: ignored

In [19]:
if __name__ == '__main__':
  x = np.array([1.0])
  y = x**2
  print(type(x),x.ndim)
  print(type(y))
  x = np.array(1.0)
  y = x**2
  print(type(x),x.ndim)
  print(type(y))

<class 'numpy.ndarray'> 1
<class 'numpy.ndarray'>
<class 'numpy.ndarray'> 0
<class 'numpy.float64'>


#### ndarray 차이
+ ndarray 의 ndim 이 1이상일 경우에는 연산을 했을 때 ndarray 로 유지되지만 0일 경우에는 float로 변환된다. 따라서 새로운 함수가 필요

In [20]:
def as_array(x):
  if np.isscalar(x):
    return np.array(x)
  return(x)
  

In [23]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        #as_array() 추가
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx
def square(x):
  f = Square()
  return f(x)

def exp(x):
  f = Exp()
  return f(x)